In [1]:
import os
import glob
import pandas as pd
import argparse
import xml.etree.ElementTree as ET
import math

def xml_to_csv(path):
    xml_list = []
    for num, xml_file in enumerate(glob.glob(path + '/*.xml')):
        print(num)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        if(root.findall('object')):
          for member in root.findall('object'):
            if(member[0].text):
              value = (root.find('filename').text,
                      int(root.find('size')[0].text),
                      int(root.find('size')[1].text),
                      member[0].text,
                      int(member[4][0].text),
                      int(member[4][1].text),
                      int(member[4][2].text),
                      int(member[4][3].text)
                      )
              xml_list.append(value)

        else:
          print("No person object detected")
          value = (root.find('filename').text,
                    int(root.find('size')[0].text),
                    int(root.find('size')[1].text),
                    None,
                    math.nan,
                    math.nan,
                    math.nan,
                    math.nan
                    )
          xml_list.append(value)
    
    column_name = [
        "filename",
        "width",
        "height",
        "class",
        "xmin",
        "ymin",
        "xmax",
        "ymax",
    ]
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# working directory
!find /content/drive/ -name UAV -type d > path.txt
file = open('path.txt', 'r')
Path = file.read()
Path = Path.strip()

In [4]:
xml_path = Path + '/Camera_infrared/resized_InfraredImg'
print(xml_path)

/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Camera_infrared/resized_InfraredImg


In [ ]:
df = xml_to_csv(xml_path)
df.to_csv(Path + '/infrared_info.csv', index = False, header = True)
print(df.head(50))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
No person object detected
17
18
19
20
21
22
23
No person object detected
24
25
No person object detected
26
27
28
29
30
31
32
No person object detected
33
34
35
36
37
No person object detected
38
39
40
41
42
43
44
45
46
47
No person object detected
48
49
50
51
52
53
54
55
56
57
No person object detected
58
59
60
61
62
63
64
65
No person object detected
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
No person object detected
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
No person object detected
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
No person object detected
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
No person object detected
158
159
No person object detected
160
161
No person object detected
162
163
164
165
166
167
168
No person object detected
169
170
171
172
173
174
175
176
177
No perso

In [5]:
%pip install tensorflow-object-detection-api

     |████████████████████████████████| 577 kB 9.3 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 3.2 MB 32.0 MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844512 sha256=af4263779388011c525b6a94894eadea71da02c43d62fbf7c1e7143761ae5d27
  Stored in directory: /root/.cache/pip/wheels/71/7e/a2/461ab817fbaef68ec9cc60df16d3669d1285f032e4c98179bf
Successfully built tensorflow-object-detection-api


In [ ]:
day_df = xml_to_csv('/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/dataset/resize_img')
day_df.to_csv('/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/dataset/resize_img/daytime_info.csv', index = False, header = True)
print(day_df.head(50))

In [9]:
import tensorflow as tf
tf.gfile = tf.io.gfile
import io
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

def split(df, group):
    data = namedtuple("data", ["filename", "object"])
    gb = df.groupby(group)
    return [
        data(filename, gb.get_group(x))
        for filename, x in zip(gb.groups.keys(), gb.groups)
    ]

def create_tf_example(group, path, label_map):
    with tf.io.gfile.GFile(os.path.join(path, "{}".format(group.filename)), "rb") as fid:  
      encoded_jpg = fid.read()
    width = 320
    height = 320

    filename = group.filename.encode("utf8")
    image_format = b"JPG"

    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
      xmins.append(row["xmin"] / width)
      xmaxs.append(row["xmax"] / width)
      ymins.append(row["ymin"] / height)
      ymaxs.append(row["ymax"] / height)
      classes_text.append(row["class"].encode("utf8"))
      class_index = label_map.get(row["class"])
      assert (
          class_index is not None
      ), "class label: `{}` not found in label_map: {}".format(
          row["class"], label_map
      )
      classes.append(class_index)

    tf_example = tf.train.Example(
        features=tf.train.Features(
            feature={
                "image/height": dataset_util.int64_feature(height),
                "image/width": dataset_util.int64_feature(width),
                "image/filename": dataset_util.bytes_feature(filename),
                "image/source_id": dataset_util.bytes_feature(filename),
                "image/encoded": dataset_util.bytes_feature(encoded_jpg),
                "image/format": dataset_util.bytes_feature(image_format),
                "image/object/bbox/xmin": dataset_util.float_list_feature(xmins),
                "image/object/bbox/xmax": dataset_util.float_list_feature(xmaxs),
                "image/object/bbox/ymin": dataset_util.float_list_feature(ymins),
                "image/object/bbox/ymax": dataset_util.float_list_feature(ymaxs),
                "image/object/class/text": dataset_util.bytes_list_feature(
                    classes_text
                ),
                "image/object/class/label": dataset_util.int64_list_feature(classes),
            }
        )
    )
    return tf_example

def main(img_path, csv_input, label_map, output_path):

    #writer = tf.compat.v1.python_io.TFRecordWriter(output_path)
    writer = tf.io.TFRecordWriter(output_path) 
    print(csv_input)
    examples = pd.read_csv(csv_input)
    examples = examples.dropna()

    # Load the `label_map` from pbtxt file.
    from object_detection.utils import label_map_util

    label_map = label_map_util.load_labelmap(label_map)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True
    )
    category_index = label_map_util.create_category_index(categories)
    label_map = {}
    for k, v in category_index.items():
        label_map[v.get("name")] = v.get("id")

    grouped = split(examples, "filename")
    for group in grouped:
        tf_example = create_tf_example(group,img_path, label_map)
        writer.write(tf_example.SerializeToString())

    writer.close()
    print("Successfully created the TFRecords: {}".format(output_path))

In [ ]:
%%bash
touch /content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/label_map_person.pbtxt

__usage="
item {
  name: \"person\"
  id: 1
}

item {
  name: \"None\"
  id: 2
}
"


echo "$__usage" > /content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/label_map_person.pbtxt
cd /content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data
cat label_map_person.pbtxt


item {
  name: "person"
  id: 1
}

item {
  name: "None"
  id: 2
}



In [10]:
imgPATH = '/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Camera_infrared/resized_InfraredImg'
inputCSV = '/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/infraredImg_info.csv'
labelMAP = '/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/label_map_person.pbtxt'
outputPATH = '/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/infraredImg_tf.tfrecord'
main(imgPATH, inputCSV, labelMAP, outputPATH)

/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/infraredImg_info.csv
Successfully created the TFRecords: /content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/infraredImg_tf.tfrecord


In [11]:
imgPATH = '/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/dataset/resize_img'
inputCSV = '/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/daytimeImg_info.csv'
labelMAP = '/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/label_map_person.pbtxt'
outputPATH = '/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/daytimeImg_tf.tfrecord'
main(imgPATH, inputCSV, labelMAP, outputPATH)

/content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/daytimeImg_info.csv
Successfully created the TFRecords: /content/drive/Othercomputers/MyLaptop/YOLO/UAV/Model/data/daytimeImg_tf.tfrecord
